In [ ]:
import torch
import torch.nn as nn

from pyfiles import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

full_model_path = f"result/VGG16_Full/model_best.pth.tar"
lr = 3e-3
epochs = 100
prune_schedule = {0: 0.4,
                  12: 0.2,
                  24: 0.1,
                  32: 0.1}
sparsity = sum(prune_schedule.values())


stype = 'os'


model_name = f"__VGG16_{stype}_prune_{sparsity:.2f}"
fdir = 'result/' + model_name

model = VGG16()
checkpoint = torch.load(full_model_path, weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
train_model(model, fdir, criterion, optimizer, epochs, prune_schedule, stype=stype)

In [ ]:
model = VGG16()
attach_prune(model)
print(f'Model param count: {count_param(model)}')

checkpoint = torch.load(f'{fdir}/model_best.pth.tar', weights_only=False)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

val_model(model)

print_sparsity(model)